# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tufanbeyli,38.2633,36.2206,275.49,35,0,0.85,TR,1709073115
1,1,blackmans bay,-43.0167,147.3167,292.33,66,100,1.72,AU,1709073116
2,2,howard springs,-12.4970,131.0470,301.41,95,62,2.79,AU,1709073117
3,3,fisterra,42.9049,-9.2629,283.01,76,100,3.98,ES,1709073118
4,4,port-aux-francais,-49.3500,70.2167,281.36,96,98,11.17,TF,1709073119


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points('Lng', 'Lat', geo = True,
                                      tiles = 'OSM', size = 'Humidity', 
                                      color = 'City', alpha = 0.5,
                                      frame_height= 500, frame_width= 650)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
cities_df = city_data_df[(city_data_df['Humidity'] < 40) & (city_data_df['Cloudiness'] == 0) &
                         (city_data_df['Wind Speed'] < 10)] 

# Drop any rows with null values
cities_df = cities_df.dropna()

# Display sample data
cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tufanbeyli,38.2633,36.2206,275.49,35,0,0.85,TR,1709073115
90,90,vicente guerrero,19.1194,-98.1708,300.81,14,0,6.01,MX,1709073102
269,269,calvinia,-31.4707,19.7760,293.13,20,0,4.66,ZA,1709073386
289,289,illizi,26.4833,8.4667,289.04,12,0,2.74,DZ,1709073404
307,307,riverside,33.7333,-116.0508,295.59,34,0,1.54,US,1709073421


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotels_df = cities_df[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotels_df['Hotel Name'] = ''

# Display sample data
hotels_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,tufanbeyli,TR,38.2633,36.2206,35,
90,vicente guerrero,MX,19.1194,-98.1708,14,
269,calvinia,ZA,-31.4707,19.7760,20,
289,illizi,DZ,26.4833,8.4667,12,
307,riverside,US,33.7333,-116.0508,34,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000

params = {
    "apiKey": geoapify_key,
    "radius": radius,
    "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotels_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotels_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotels_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotels_df.loc[index, 'City']} - nearest hotel: {hotels_df.loc[index, 'Hotel Name']}")

# Display sample data
hotels_df

Starting hotel search
tufanbeyli - nearest hotel: No hotel found
vicente guerrero - nearest hotel: HOTEL SNAYLF
calvinia - nearest hotel: No hotel found
illizi - nearest hotel: بوناقة
riverside - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
pinarbasi - nearest hotel: Aziziye
medina - nearest hotel: The Oberoi Madina
rikaze - nearest hotel: Gesar Hotel
roma - nearest hotel: Wingate by Wyndham Rome
salmas - nearest hotel: هتل آتا
ciudad de huajuapan de leon - nearest hotel: Casa Blanca
tindouf - nearest hotel: محمد بوسبي
enoch - nearest hotel: Travelodge by Wyndham Cedar City
awjilah - nearest hotel: No hotel found
gebeit - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
0,tufanbeyli,TR,38.2633,36.2206,35,No hotel found
90,vicente guerrero,MX,19.1194,-98.1708,14,HOTEL SNAYLF
269,calvinia,ZA,-31.4707,19.7760,20,No hotel found
289,illizi,DZ,26.4833,8.4667,12,بوناقة
307,riverside,US,33.7333,-116.0508,34,No hotel found
360,laguna,US,38.4210,-121.4238,37,Holiday Inn Express & Suites
364,pinarbasi,TR,38.7229,36.3931,33,Aziziye
381,medina,SA,24.4686,39.6142,15,The Oberoi Madina
382,rikaze,CN,29.2500,88.8833,26,Gesar Hotel
398,roma,US,43.2128,-75.4557,39,Wingate by Wyndham Rome


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot = hotels_df.hvplot.points('Lng', 'Lat', geo = True,
                                      tiles = 'OSM', size = 'Humidity', 
                                      color = 'City', alpha = 0.5,
                                      frame_height= 500, frame_width= 650, hover_cols = ['Hotel Name','Country'])

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)